# 0) Introduction:

In [7]:
print('Assignment1 made by Andre Llumiquinga for ')
print("End-to-end Data Science lifecycle project showcasing structured project organization, reproducible workflows, and a modular data processing pipeline.")

Assignment1 made by Andre Llumiquinga for 
End-to-end Data Science lifecycle project showcasing structured project organization, reproducible workflows, and a modular data processing pipeline.


# 1) Setup paths:

In [8]:
import os
import sys
import platform
import json
import random
from pathlib import Path

import numpy as np
import pandas as pd

print('Python:', sys.version.split()[0])
print('Platform:', platform.platform())
print('pandas:', pd.__version__)

Python: 3.13.7
Platform: Windows-11-10.0.26100-SP0
pandas: 2.3.3


In [9]:
from pathlib import Path

PROJECT_ROOT = Path.cwd() / "assignment1"
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_INTERIM = PROJECT_ROOT / "data" / "interim"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
REPORTS = PROJECT_ROOT / "reports"

for p in [DATA_RAW, DATA_INTERIM, DATA_PROCESSED, REPORTS]:
    p.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT: ")
PROJECT_ROOT

PROJECT_ROOT: 


WindowsPath('C:/Users/DyFer/Documents/ELTE/2Semester/ITDS/A01/data-processing-pipeline/assignment1')

In [10]:
# 3.1 Freeze a minimal requirements file (teaching demo)
requirements = [
    f"pandas=={pd.__version__}",
    f"numpy=={np.__version__}",
    "openpyxl"  # needed for reading/writing .xlsx with pandas
]
(PROJECT_ROOT / "requirements.txt").write_text("\n".join(requirements) + "\n")
print((PROJECT_ROOT / "requirements.txt").read_text())

pandas==2.3.3
numpy==2.3.4
openpyxl



# 2) Load:

In [11]:
# --- Dataset filename  ---
raw_csv_filename = "synthetic_dataset.csv"

# --- Build full path using DATA_RAW ---
raw_csv_path = DATA_RAW / raw_csv_filename

# --- Read CSV ---
df_csv = pd.read_csv(raw_csv_path)

print("Data Frame loaded done!")

Data Frame loaded done!


# 3) Check :

In [12]:
# --- Basic checks ---
print("===== SHAPE =====")
print(df_csv.shape)

print("\n===== HEAD =====")
print(df_csv.head())

print("\n===== INFO =====")
df_csv.info()

print("\n===== MISSING VALUES SUMMARY =====")
missing_summary = df_csv.isna().sum().to_frame(name="missing_count")
missing_summary["missing_percentage"] = (
    missing_summary["missing_count"] / len(df_csv) * 100
)
print(missing_summary)

print("\n===== DUPLICATES COUNT =====")
duplicates_count = df_csv.duplicated().sum()
print("Number of duplicate rows:", duplicates_count)


===== SHAPE =====
(4362, 5)

===== HEAD =====
  Category   Price    Rating         Stock  Discount
0      NaN  5548.0  1.870322           NaN       0.0
1      NaN  3045.0  4.757798           NaN      38.0
2      NaN  4004.0       NaN      In Stock       0.0
3      NaN  4808.0  1.492085           NaN      33.0
4      NaN  1817.0       NaN  Out of Stock      23.0

===== INFO =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4362 entries, 0 to 4361
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  1614 non-null   object 
 1   Price     4188 non-null   float64
 2   Rating    2312 non-null   float64
 3   Stock     3010 non-null   object 
 4   Discount  3970 non-null   float64
dtypes: float64(3), object(2)
memory usage: 170.5+ KB

===== MISSING VALUES SUMMARY =====
          missing_count  missing_percentage
Category           2748           62.998624
Price               174            3.988996
Rating           

# 4) Data Preparation :

## 4.1) Create function for Data Cleaning :

In [13]:
def clean_mydata(df: pd.DataFrame) -> pd.DataFrame:
    # --- Make a copy to avoid modifying original ---
    df = df.copy()

    # 1️⃣ Rename columns (lowercase + snake_case style)
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )

    # 2️⃣ Remove rows with missing values
    df = df.dropna()

    # 3️⃣ Drop exact duplicate rows
    df = df.drop_duplicates()

    return df

print('Clean function created!')

Clean function created!


## 4.2) Create function for Data Transformation:

In [14]:
def transformation_mydata(df: pd.DataFrame) -> pd.DataFrame:
    # --- Make a copy to avoid modifying original ---
    df = df.copy()

    # 4️⃣ Create a derived column: final_price after discount
    # (assuming discount is percentage)
    df["final_price"] = df["price"] * (1 - df["discount"] / 100)

    # 5️⃣ Sort alphabetically by category
    df = df.sort_values(by="category")

    # 6️⃣ Reset index
    df = df.reset_index(drop=True)

    return df

print('Transformation function created!')

Transformation function created!


## 4.3) Execute and Validate Clean function :

In [15]:
# --- Before cleaning ---
print("===== BEFORE CLEANING =====")
print("Shape:", df_csv.shape)
print("Missing values:\n", df_csv.isna().sum())
print("Duplicates:", df_csv.duplicated().sum())

# --- Apply cleaning function ---
df_clean = clean_mydata(df_csv)
dataset_name = "synthetic_dataset"
interim_path = DATA_INTERIM / f"{dataset_name}_interim.csv"
df_clean.to_csv(interim_path, index=False)
print("\nInterim dataset saved to:", interim_path)

# --- After cleaning ---
print("\n===== AFTER CLEANING =====")
print("Shape:", df_clean.shape)
print("Missing values:\n", df_clean.isna().sum())
print("Duplicates:", df_clean.duplicated().sum())

===== BEFORE CLEANING =====
Shape: (4362, 5)
Missing values:
 Category    2748
Price        174
Rating      2050
Stock       1352
Discount     392
dtype: int64
Duplicates: 15

Interim dataset saved to: C:\Users\DyFer\Documents\ELTE\2Semester\ITDS\A01\data-processing-pipeline\assignment1\data\interim\synthetic_dataset_interim.csv

===== AFTER CLEANING =====
Shape: (540, 5)
Missing values:
 category    0
price       0
rating      0
stock       0
discount    0
dtype: int64
Duplicates: 0


## 4.4) Execute Transformation function :

In [16]:
df_transform = transformation_mydata(df_clean)

dataset_name = "synthetic_dataset"
processed_path = DATA_PROCESSED / f"{dataset_name}_processed.csv"
df_transform.to_csv(processed_path, index=False)

print("\n===== TRANFORMED DATA HEAD =====")
print(df_transform.head())


===== TRANFORMED DATA HEAD =====
  category   price    rating         stock  discount  final_price
0        A  7596.0  2.799564      In Stock      10.0      6836.40
1        A  9116.0  2.160278      In Stock      45.0      5013.80
2        A  4951.0  4.883903  Out of Stock      23.0      3812.27
3        A  9854.0  2.694787  Out of Stock       3.0      9558.38
4        A  9093.0  4.316502      In Stock       9.0      8274.63


# 5) Save:

In [17]:
# --- Create summary table grouped by category ---
summary_table = (
    df_transform
    .groupby("category")
    .agg(
        total_products=("category", "count"),
        avg_price=("price", "mean"),
        avg_final_price=("final_price", "mean"),
        avg_rating=("rating", "mean"),
        avg_discount=("discount", "mean"),
    )
    .reset_index()
)

# Optional: round numeric values for cleaner reporting
summary_table = summary_table.round(2)

summary_path = REPORTS / "summary_table.csv"

summary_table.to_csv(summary_path, index=False)

print("Summary table saved to:", summary_path)
print(summary_table)

Summary table saved to: C:\Users\DyFer\Documents\ELTE\2Semester\ITDS\A01\data-processing-pipeline\assignment1\reports\summary_table.csv
  category  total_products  avg_price  avg_final_price  avg_rating  \
0        A             130    4975.52          3727.09        3.07   
1        B             143    5222.77          3952.31        3.04   
2        C             131    5234.42          3834.47        3.00   
3        D             136    5260.48          3947.12        3.14   

   avg_discount  
0         24.00  
1         23.80  
2         26.82  
3         25.43  
